# Deep learning 1




| Exercise | Requires coding | What to code (if anything) | What should you learn (or practice)    | Done it! |
|----------|-----------------|----------------------------|----------------------------------------|----------|
| 5.1      | Yes             |    Backpropagation                         | Backprop details                           | No       |
| 5.2      | No              |                            | Familiarize with theano                           | No       |
| 5.3      |   Yes         |   Forward pass MLP                            | Symbolic forward pass                                | No       |
| 5.4      |   No         |        | check TheanoMLP  class                           | No       |
| 5.5      |   No         |        | Data in theano (shared variables)                                | No       |
| 5.6      |   No         |        | Compare Numpy vs Theano                                | No       |








In [238]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import sys
sys.path.append('../../lxmls-toolkit')
import lxmls
import scipy
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Ecxercise 5.1

**Start by loading Amazon sentiment corpus used in day 1**

- Go to **lxmls/deep\_learning/mlp.py** look at the class NumpyMLP and the function ** grads( )**. 

- Complete the code of the NumpyMLP class with the Backpropagation recursion that explained in the guide.

- Once you are done. Try different network geometries by increasing the number of layers and layer sizes...


#### Important

The code is build to accept data as columns. 

For example a minibatch with 10 examples should have shape `n_features x 10`

In [307]:
import numpy as np
import lxmls.readers.sentiment_reader as srs
scr = srs.SentimentCorpus("books")
train_x = scr.train_X.T
train_y = scr.train_y[:, 0]
test_x = scr.test_X.T
test_y = scr.test_y[:, 0]

train_x = train_x.astype("float32")
train_y = train_y.astype("int32")
test_x = test_x.astype("float32")
test_y = test_y.astype("int32")

2000
1600


In [308]:
print "unique classes in the data: ", np.unique(train_y)
print "train shape: ", train_x.shape
print "test shape: ", train_x.shape

unique classes in the data:  [0 1]
train shape:  (13989, 1600)
test shape:  (13989, 1600)


In [309]:
# Neural network modules
import lxmls.deep_learning.mlp as dl
import lxmls.deep_learning.sgd as sgd

# Model parameters
geometry = [train_x.shape[0], 100, 2]
actvfunc = ['sigmoid', 'softmax']

# Instantiate model
mlp = dl.NumpyMLP(geometry, actvfunc) 

In [310]:
print "number of layers: ", mlp.n_layers

number of layers:  2


In [311]:
[x.shape for x in mlp.params]

[(100, 13989), (100, 1), (2, 100), (2, 1)]


#### About mlp.forward 

the function `mlp.forward()` returns the probability of each of the classes for each example in the minibatch as a column vector. 

For example, mlp.forward(train_x[:,0:3]) should return 

```
[p(y|x[:,1]), p(y|x[:,2]), p(y|x[:,3])]
```

which will look like

```
array([[ 0.71735224,  0.11738626,  0.93179871],
       [ 0.28264776,  0.88261374,  0.06820129]])
```



In [312]:
mlp.forward(train_x[:,0:3])

array([[ 0.33014659,  0.53808734,  0.84166145],
       [ 0.66985341,  0.46191266,  0.15833855]])

#### About the gradient shapes

In [313]:
mlp.forward(train_x[:,0:3])

array([[ 0.33014659,  0.53808734,  0.84166145],
       [ 0.66985341,  0.46191266,  0.15833855]])

In [314]:
print mlp.grads(train_x[:,0:5], train_y[0:5])[0].shape
print mlp.grads(train_x[:,0:5], train_y[0:5])[1].shape
print mlp.grads(train_x[:,0:5], train_y[0:5])[2].shape
print mlp.grads(train_x[:,0:5], train_y[0:5])[3].shape

(100, 13989)
(100, 1)
(2, 100)
(2, 1)


#### Train the model

In [316]:
# Model parameters
geometry = [train_x.shape[0], 100, 2]
actvfunc = ['sigmoid', 'softmax']

# Instantiate model
mlp = dl.NumpyMLP(geometry, actvfunc) 

# Model parameters
n_iter = 5
bsize  = 25
lrate  = 0.01

# Train
sgd.SGD_train(mlp, n_iter, bsize=bsize, lrate=lrate, train_set=(train_x, train_y))

Batch 64/64 (100%)   Epoch  1/ 5 in 23.08 seg
Batch 64/64 (100%)   Epoch  2/ 5 in 22.75 seg
Batch 64/64 (100%)   Epoch  3/ 5 in 22.81 seg
Batch 64/64 (100%)   Epoch  4/ 5 in 23.33 seg
Batch 64/64 (100%)   Epoch  5/ 5 in 23.40 seg
 


In [319]:
acc_train = sgd.class_acc(mlp.forward(train_x), train_y)[0]
acc_test = sgd.class_acc(mlp.forward(test_x), test_y)[0]

print "MLP Architecture: (%s)" % (geometry)
print "Amazon Sentiment Accuracy train: %f test: %f" % ( acc_train, acc_test)

MLP Architecture: ([13989, 100, 2])
Amazon Sentiment Accuracy train: 0.850000 test: 0.737500


## Exercise 5.2 
#### Begin Exercise 5.2

Get in contact with Theano. Learn the difference between a symbolic
representation and a function. Start by implementing the first layer of our
previous MLP in Numpy 

In [320]:
# Numpy code
x        = test_x             # Test set 
W1, b1   = mlp.params[:2]     # Weights and bias of fist layer 
z1       = np.dot(W1, x) + b1 # Linear transformation
tilde_z1 = 1/(1+np.exp(-z1))  # Non-linear transformation  

In [321]:
# Theano code. 
# NOTE: We use undescore to denote symbolic equivalents to Numpy variables. 
# This is no Python convention!.
import theano
import theano.tensor as T
_x = T.matrix('x')

Note that this variable does not have any particular value, nor a space
reserved in memory for it. It contains just a symbolic definition of what the
variable can contain. The particular values will be given when we use it to
compile a function. 

We could actually use the same definition format to define the weights and give
their particular values as inputs to the compiled function. However, since we
will be using a more complicated format in later exercises, we will use it here
as well. The **```shared```** class allows to define variables that are shared
across functions. They are also given a concrete value so that we do not need
to give it for each function call. This format is therefore ideal for the
weights of our network.

In [322]:
_W1 = theano.shared(value=W1, name='W1', borrow=True) 
_b1 = theano.shared(value=b1, name='b1', borrow=True, broadcastable=(False, True)) 

Now lets describe the operations we want to do with the variables. Again only
symbolically. This is done by replacing our usual operations by Theano symbolic
ones when necessary e. g. the internal product dot() or the sigmoid. Some
operations like e.g. $+$ are automatically recognized by Theano (operator
overloading). 

In [323]:
_z1            = T.dot(_W1, _x) + _b1
_tilde_z1      = T.nnet.sigmoid(_z1)
# Keep in mind that naming variables is useful when debugging
_z1.name       = 'z1'
_tilde_z1.name = 'tilde_z1'

When debugging the code it is often useful to print the graph of computations.

In [324]:
# Perceptron computation graph
theano.printing.debugprint(_tilde_z1)


sigmoid [id A] 'tilde_z1'   
 |Elemwise{add,no_inplace} [id B] 'z1'   
   |dot [id C] ''   
   | |W1 [id D]
   | |x [id E]
   |b1 [id F]


It is important to keep in mind that, until this point, we do not have a
function we can use to produce any practical input. In order to obtain this we
have to compile this function by calling    

In [325]:
layer1 = theano.function([_x], _tilde_z1)

Note the use of [ ] for the input variables, even if we just specify one
variable. We can now do a test to compare the Numpy and Theano implementations
and see that they give the same outputs.

In [326]:
# Check Numpy and Theano match
if np.allclose(tilde_z1, layer1(x.astype(theano.config.floatX))):
    print "\nNumpy and Theano Perceptrons are equivalent"
else:
    raise ValueError, "Numpy and Theano Perceptrons are different"


Numpy and Theano Perceptrons are equivalent


#### End exercise 5.2

#### Symbolic forward pass
In the previous section you have seen how to create symbolic Theano functions
with shared parameters. You have thus all you need to implement the whole
forward pass of a generic MLP in Theano.


## Exercise 5.3

#### Begin Exercise 5.3

Complete the method **```_forward()```** inside of the **```lxmls/deep\_learning/mlp.py```**, in the class TheanoMLP.

Note that this is called only once at the initialization of the
class. To debug your implementation put a breakpoint at the \_\_init\_\_
function call. Hint: Note that this is very similar to NumpyMLP.forward().
You just need to keep track of the symbolic variable representing the output of
the network after each layer is applied and compile the function at the end.
After you are finished instantiate a Theano class and check that Numpy and
Theano forward pass are the same. 


In [327]:
from lxmls.deep_learning import mlp 

In [328]:
mlp_a = mlp.NumpyMLP(geometry, actvfunc)
mlp_b = mlp.TheanoMLP(geometry, actvfunc)

#### End Exercise 5.3


## Symbolic differentiation
In the previous section we compiled the forward pass of a MLP. In this section
we will do the same with the cost used for training. We will also derive the
gradients although this will be trivial once we have the cost function compiled.     

## Exercise 5.4

We first see an example that does not use any of the code in TheanoMLP but
rather continues from what you wrote in exercise 6.3. In this exercise you
completed a sigmoid layer with Theano. To get some values for the weights we
used the first layer of the network you trained in 6.2. now we are going to use
the second layer as well. This is thus assuming that your network in 6.2 has
only two layers e.g. the recommended geometry (I, 20, 2). Make sure this is the
case before starting this exercise.  

For the sake of clarity, lets write here the part of Ex. 5.2 that we had completed

In [329]:
# Get the values from our MLP from Ex 6.2
W1, b1   = mlp_a.params[:2]     # Weights and bias of fist layer 
# First layer symbolic variables
_x  = T.matrix('x')
_W1 = theano.shared(value=W1, name='W1', borrow=True) 
_b1 = theano.shared(value=b1, name='b1', borrow=True, broadcastable=(False, True)) 
# First layer symbolic expressions
_z1       = T.dot(_W1, _x) + _b1
_tilde_z1 = T.nnet.sigmoid(_z1)

# OPTIONAL: If you don't want gradients...

## Optional:  Feedback aligment

The following class is build to learn using direct feedback aligment. This is not in the guide.

In [330]:
from scipy.misc import logsumexp


def index2onehot(index, N):
    """
    Transforms index to one-hot representation, for example

    Input: e.g. index = [1, 2, 0], N = 4
    Output:     [[0, 1, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0]]
    """
    L = index.shape[0]
    onehot = np.zeros((N, L))
    for l in np.arange(L):
        onehot[index[l], l] = 1
    return onehot

class NumpyMLP_nobackprop:
    """
    Basic MLP with forward-pass and non_gradient computation
    """
    
    def __init__(self, geometry, actvfunc, rng=None, model_file=None):
        """
        Input: geometry  tuple with sizes of layer

        Input: actvfunc  list of strings indicating the type of activation
                         function. Supported 'sigmoid', 'softmax'

        Input: rng       string inidcating random seed
        """

        # Generate random seed if not provided
        if rng is None:
            rng = np.random.RandomState(1234)

        # CHECK THE PARAMETERS ARE VALID
        self.sanity_checks(geometry, actvfunc)

        # THIS DEFINES THE MLP
        self.n_layers = len(geometry) - 1
        if model_file:
            if geometry or actvfunc:
                raise ValueError("If you load a model geometry and actvfunc"
                                 "should be None")
            self.params, self.actvfunc = self.load(model_file)
        else:
            # Parameters are stored as [weight0, bias0, weight1, bias1, ... ]
            # for consistency with the theano way of storing parameters
            self.params = self.init_weights(rng, geometry, actvfunc)
            self.actvfunc = actvfunc

    def forward(self, x, all_outputs=False):
        """
        Forward pass

        all_outputs = True  return intermediate activations
        """

        # This will store activations at each layer and the input. This is
        # needed to compute backpropagation
        if all_outputs:
            activations = []

            # Input
        tilde_z = x

        for n in range(self.n_layers):

            # Get weigths and bias of the layer (even and odd positions)
            W = self.params[2*n]
            b = self.params[2*n+1]

            # Linear transformation
            z = np.dot(W, tilde_z) + b

            # Non-linear transformation
            if self.actvfunc[n] == "sigmoid":
                tilde_z = 1.0 / (1+np.exp(-z))

            elif self.actvfunc[n] == "softmax":
                # Softmax is computed in log-domain to prevent
                # underflow/overflow
                tilde_z = np.exp(z - logsumexp(z, 0))

            if all_outputs:
                activations.append(tilde_z)

        if all_outputs:
            tilde_z = activations

        return tilde_z
        

    def grads(self, x, y):
        """
       Computes the gradients of the network with respect to cross entropy
       error cost
       """

        # Run forward and store activations for each layer
        activations = self.forward(x, all_outputs=True)

        # For each layer in reverse store the gradients for each parameter
        nabla_params = [None] * (2*self.n_layers)

        for n in np.arange(self.n_layers-1, -1, -1):
            # Get weigths and bias (always in even and odd positions)
            # Note that sometimes we need the weight from the next layer
            W = self.params[2*n]
            b = self.params[2*n+1]
            if n != self.n_layers-1:
                F_mat = self.feedback_matrix[2*(n+1)]

            if n == self.n_layers-1:
                # NOTE: This assumes cross entropy cost
                if self.actvfunc[n] == 'sigmoid':
                    e = (activations[n]-y) / y.shape[0]
                elif self.actvfunc[n] == 'softmax':
                    I = index2onehot(y, W.shape[0])
                    e = (activations[n]-I) / y.shape[0]
            else:
                e = np.dot(F_mat, e)
                # This is correct but confusing n+1 is n in the guide
                e *= activations[n] * (1-activations[n])

            # Weight gradient
            nabla_W = np.zeros(W.shape)
            for l in np.arange(e.shape[1]):
                if n == 0:
                    # For the first layer, the activation is the input
                    nabla_W += np.outer(e[:, l], x[:, l])
                else:
                    nabla_W += np.outer(e[:, l], activations[n-1][:, l])
            # Bias gradient
            nabla_b = np.sum(e, 1, keepdims=True)

            # Store the gradients
            nabla_params[2*n] = nabla_W
            nabla_params[2*n+1] = nabla_b

        return nabla_params

    def init_weights(self, rng, geometry, actvfunc):
        """
       Following theano tutorial at
       http://deeplearning.net/software/theano/tutorial/
       """
        params = []
        self.feedback_matrix = []
        for n in range(self.n_layers):
            n_in, n_out = geometry[n:n+2]
            weight = rng.uniform(low=-np.sqrt(6./(n_in+n_out)),
                                 high=np.sqrt(6./(n_in+n_out)),
                                 size=(n_out, n_in))
            if actvfunc[n] == 'sigmoid':
                weight *= 4
            elif actvfunc[n] == 'softmax':
                weight *= 4
            bias = np.zeros((n_out, 1))
            # Append parameters
            params.append(weight)
            params.append(bias)
            self.feedback_matrix.append( np.random.uniform(-0.5, 0.5, weight.T.shape) )
            self.feedback_matrix.append( np.random.uniform(-0.5, 0.5, bias.shape) )

        return params

    def sanity_checks(self, geometry, actvfunc):
        # CHECK ACTIVATIONS
        if actvfunc:
            # Supported actvfunc
            supported_acts = ['sigmoid', 'softmax']
            if geometry and (len(actvfunc) != len(geometry)-1):
                raise ValueError("The number of layers and actvfunc does not match")
            elif any([act not in supported_acts for act in actvfunc]):
                raise ValueError("Only these actvfunc supported %s" % (" ".join(supported_acts)))
            # All internal layers must be a sigmoid
            for internal_act in actvfunc[:-1]:
                if internal_act != 'sigmoid':
                    raise ValueError("Intermediate layers must be sigmoid")

    def save(self, model_path):
        """
        Save model
        """
        par = self.params + self.actvfunc
        with open(model_path, 'wb') as fid:
            cPickle.dump(par, fid, cPickle.HIGHEST_PROTOCOL)

    def load(self, model_path):
        """
        Load model
        """
        with open(model_path) as fid:
            par = cPickle.load(fid, cPickle.HIGHEST_PROTOCOL)
            params = par[:len(par)//2]
            actvfunc = par[len(par)//2:]
        return params, actvfunc

    def plot_weights(self, show=True, aspect='auto'):
        """
       Plots the weights of the newtwork
       """
        import matplotlib.pyplot as plt
        plt.figure()
        for n in range(self.n_layers):
            # Get weights
            W = self.params[2*n]
            b = self.params[2*n+1]

            plt.subplot(2, self.n_layers, n+1)
            plt.imshow(W, aspect=aspect, interpolation='nearest')
            plt.title('Layer %d Weight' % n)
            plt.colorbar()
            plt.subplot(2, self.n_layers, self.n_layers+(n+1))
            plt.plot(b)
            plt.title('Layer %d Bias' % n)
            plt.colorbar()
        if show:
            plt.show()

In [331]:
# Model parameters
geometry = [train_x.shape[0], 100, 2]
actvfunc = ['sigmoid', 'softmax']

# Instantiate model
mlp = NumpyMLP_nobackprop(geometry, actvfunc) 

In [332]:
acc_train = sgd.class_acc(mlp.forward(train_x), train_y)[0]
acc_test = sgd.class_acc(mlp.forward(test_x), test_y)[0]

print "MLP Architecture: (%s)" % (geometry)
print "Amazon Sentiment Accuracy train: %f test: %f" % ( acc_train, acc_test)

MLP Architecture: ([13989, 100, 2])
Amazon Sentiment Accuracy train: 0.542500 test: 0.532500


In [333]:
# Train
sgd.SGD_train(mlp, n_iter=5, bsize=bsize, lrate=0.5, train_set=(train_x, train_y))

Batch 64/64 (100%)   Epoch  1/ 5 in 21.83 seg
Batch 64/64 (100%)   Epoch  2/ 5 in 22.02 seg
Batch 64/64 (100%)   Epoch  3/ 5 in 21.32 seg
Batch 64/64 (100%)   Epoch  4/ 5 in 21.69 seg
Batch 64/64 (100%)   Epoch  5/ 5 in 21.54 seg
 


In [334]:
acc_train = sgd.class_acc(mlp.forward(train_x), train_y)[0]
acc_test = sgd.class_acc(mlp.forward(test_x), test_y)[0]

print "MLP Architecture: (%s)" % (geometry)
print "Amazon Sentiment Accuracy train: %f test: %f" % ( acc_train, acc_test)

MLP Architecture: ([13989, 100, 2])
Amazon Sentiment Accuracy train: 0.911875 test: 0.770000


## Optional:  Direct feedback aligment

We can actually update the weights in parallel


In [335]:
from scipy.misc import logsumexp

def index2onehot(index, N):
    """
    Transforms index to one-hot representation, for example

    Input: e.g. index = [1, 2, 0], N = 4
    Output:     [[0, 1, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0]]
    """
    L = index.shape[0]
    onehot = np.zeros((N, L))
    for l in np.arange(L):
        onehot[index[l], l] = 1
    return onehot

class NumpyMLP_dfa:
    """
    Basic MLP with forward-pass and non_gradient computation
    """
    
    def __init__(self, geometry, actvfunc, rng=None, model_file=None):
        """
        Input: geometry  tuple with sizes of layer

        Input: actvfunc  list of strings indicating the type of activation
                         function. Supported 'sigmoid', 'softmax'

        Input: rng       string inidcating random seed
        """

        # Generate random seed if not provided
        if rng is None:
            rng = np.random.RandomState(1234)

        # CHECK THE PARAMETERS ARE VALID
        self.sanity_checks(geometry, actvfunc)

        # THIS DEFINES THE MLP
        self.n_layers = len(geometry) - 1
        if model_file:
            if geometry or actvfunc:
                raise ValueError("If you load a model geometry and actvfunc"
                                 "should be None")
            self.params, self.actvfunc = self.load(model_file)
        else:
            # Parameters are stored as [weight0, bias0, weight1, bias1, ... ]
            # for consistency with the theano way of storing parameters
            self.params = self.init_weights(rng, geometry, actvfunc)
            self.actvfunc = actvfunc

    def forward(self, x, all_outputs=False):
        """
        Forward pass

        all_outputs = True  return intermediate activations
        """

        # This will store activations at each layer and the input. This is
        # needed to compute backpropagation
        if all_outputs:
            activations = []

            # Input
        tilde_z = x

        for n in range(self.n_layers):

            # Get weigths and bias of the layer (even and odd positions)
            W = self.params[2*n]
            b = self.params[2*n+1]

            # Linear transformation
            z = np.dot(W, tilde_z) + b

            # Non-linear transformation
            if self.actvfunc[n] == "sigmoid":
                tilde_z = 1.0 / (1+np.exp(-z))

            elif self.actvfunc[n] == "softmax":
                # Softmax is computed in log-domain to prevent
                # underflow/overflow
                tilde_z = np.exp(z - logsumexp(z, 0))

            if all_outputs:
                activations.append(tilde_z)

        if all_outputs:
            tilde_z = activations

        return tilde_z
        

    def grads(self, x, y):
        """
       Computes the gradients of the network with respect to cross entropy
       error cost
       """

        # Run forward and store activations for each layer
        activations = self.forward(x, all_outputs=True)

        # For each layer in reverse store the gradients for each parameter
        nabla_params = [None] * (2*self.n_layers)

        for n in np.arange(self.n_layers-1, -1, -1):

            # Get weigths and bias (always in even and odd positions)
            # Note that sometimes we need the weight from the next layer
            W = self.params[2*n]
            b = self.params[2*n+1]
            if n != self.n_layers-1:
                F_mat = self.feedback_matrix[2*(n+1)]

            # NOTE: This assumes cross entropy cost
            if self.actvfunc[n] == 'sigmoid':
                e = (activations[n]-y) / y.shape[0]
            elif self.actvfunc[n] == 'softmax':
                I = index2onehot(y, W.shape[0])
                e = (activations[n]-I) / y.shape[0]


            # Weight gradient
            nabla_W = np.zeros(W.shape)
            for l in np.arange(e.shape[1]):
                if n == 0:
                    # For the first layer, the activation is the input
                    nabla_W += np.outer(e[:, l], x[:, l])
                else:
                    nabla_W += np.outer(e[:, l], activations[n-1][:, l])
            # Bias gradient
            nabla_b = np.sum(e, 1, keepdims=True)

            # Store the gradients
            nabla_params[2*n] = nabla_W
            nabla_params[2*n+1] = nabla_b

        return nabla_params

    def init_weights(self, rng, geometry, actvfunc):
        """
        Following theano tutorial at
        http://deeplearning.net/software/theano/tutorial/
        """
        params = []
        self.feedback_matrix = []
        n_classes = geometry[-1]
        
        for n in range(self.n_layers):
            n_in, n_out = geometry[n:n+2]
            weight = rng.uniform(low=-np.sqrt(6./(n_in+n_out)),
                                 high=np.sqrt(6./(n_in+n_out)),
                                 size=(n_out, n_in))
            if actvfunc[n] == 'sigmoid':
                weight *= 4
            elif actvfunc[n] == 'softmax':
                weight *= 4
            bias = np.zeros((n_out, 1))
            # Append parameters
            params.append(weight)
            params.append(bias)
            self.feedback_matrix.append( np.random.uniform(-0.5, 0.5, (n_classes, weight.T.shape[1]) ) )
            self.feedback_matrix.append( np.random.uniform(-0.5, 0.5, bias.shape) )

        return params

    def sanity_checks(self, geometry, actvfunc):

        # CHECK ACTIVATIONS
        if actvfunc:
            # Supported actvfunc
            supported_acts = ['sigmoid', 'softmax']
            if geometry and (len(actvfunc) != len(geometry)-1):
                raise ValueError("The number of layers and actvfunc does not match")
            elif any([act not in supported_acts for act in actvfunc]):
                raise ValueError("Only these actvfunc supported %s" % (" ".join(supported_acts)))
            # All internal layers must be a sigmoid
            for internal_act in actvfunc[:-1]:
                if internal_act != 'sigmoid':
                    raise ValueError("Intermediate layers must be sigmoid")

    def save(self, model_path):
        """
        Save model
        """
        par = self.params + self.actvfunc
        with open(model_path, 'wb') as fid:
            cPickle.dump(par, fid, cPickle.HIGHEST_PROTOCOL)

    def load(self, model_path):
        """
        Load model
        """
        with open(model_path) as fid:
            par = cPickle.load(fid, cPickle.HIGHEST_PROTOCOL)
            params = par[:len(par)//2]
            actvfunc = par[len(par)//2:]
        return params, actvfunc

    def plot_weights(self, show=True, aspect='auto'):
        """
       Plots the weights of the newtwork
       """
        import matplotlib.pyplot as plt
        plt.figure()
        for n in range(self.n_layers):
            # Get weights
            W = self.params[2*n]
            b = self.params[2*n+1]

            plt.subplot(2, self.n_layers, n+1)
            plt.imshow(W, aspect=aspect, interpolation='nearest')
            plt.title('Layer %d Weight' % n)
            plt.colorbar()
            plt.subplot(2, self.n_layers, self.n_layers+(n+1))
            plt.plot(b)
            plt.title('Layer %d Bias' % n)
            plt.colorbar()
        if show:
            plt.show()

In [336]:
# Model parameters
geometry = [train_x.shape[0], 100, 2]
actvfunc = ['sigmoid', 'softmax']

# Instantiate model
mlp =  NumpyMLP_dfa(geometry, actvfunc) 

In [337]:
acc_train = sgd.class_acc(mlp.forward(train_x), train_y)[0]
acc_test = sgd.class_acc(mlp.forward(test_x), test_y)[0]

print "MLP Architecture: (%s)" % (geometry)
print "Amazon Sentiment Accuracy train: %f test: %f" % ( acc_train, acc_test)

MLP Architecture: ([13989, 100, 2])
Amazon Sentiment Accuracy train: 0.542500 test: 0.532500


In [338]:
# Train
sgd.SGD_train(mlp, n_iter=5, bsize=bsize, lrate=0.5, train_set=(train_x, train_y))

Batch 64/64 (100%)   Epoch  1/ 5 in 23.40 seg
Batch 64/64 (100%)   Epoch  2/ 5 in 22.85 seg
Batch 64/64 (100%)   Epoch  3/ 5 in 22.64 seg
Batch 64/64 (100%)   Epoch  4/ 5 in 22.77 seg
Batch 64/64 (100%)   Epoch  5/ 5 in 23.43 seg
 


In [339]:
acc_train = sgd.class_acc(mlp.forward(train_x), train_y)[0]
acc_test = sgd.class_acc(mlp.forward(test_x), test_y)[0]

print "MLP Architecture: (%s)" % (geometry)
print "Amazon Sentiment Accuracy train: %f test: %f" % ( acc_train, acc_test)

MLP Architecture: ([13989, 100, 2])
Amazon Sentiment Accuracy train: 0.996250 test: 0.840000
